# Simle Quantum Circuit Library Guide
## Introduction:
This jupyter notebook explains the fundamentals of using this quantum circuit library step by step. 

## Prerequisites:
This library uses numpy. If numpy is not installed on your system you can easily install numpy by using the following command:

`pip install numpy`

## Qubit class
This class constructs a single qubit. The constructer of this class is given two complex numbers, alpha and beta, which are the amplitudes of the zero state and one state respectively. 
Let's construct a zero state ,a one state qubits and a qubit with equal amplitudes:

In [92]:
from qubit import Qubit
import numpy as np

q0 = Qubit(alpha=1,beta=0)
q1 = Qubit(alpha=0,beta=1)
q2 = Qubit(alpha= 1/np.sqrt(2),beta= 1/np.sqrt(2))

Note that the amplitudes have to respect the normalization of the quantum state otherwise the class will throw an error.

Now let's print the qubit in different forms to show the quantum state of the qubit:

In [93]:
# Tensor form more readable:
q0.print_qubit()
q1.print_qubit()
q2.print_qubit()
# Vector form for debugging and backend of the library.
q0.print_vector_form()
q1.print_vector_form()
q2.print_vector_form()

Qubit state is 1.00|0⟩
Qubit state is 1.00|1⟩
Qubit state is 0.71|0⟩ + 0.71|1⟩
[1 0]
[0 1]
[0.70710678 0.70710678]


We can change the amplitudes using the set amplitudes method. Remember to respect the normalization condition.


In [94]:
q2.set_amplitudes(alpha=1,beta=0)

There are also getter function for the amplitudes and for the qubit vector.

In [95]:
print(q2.get_alpha())
print(q2.get_beta())
print(q2.get_vector())

1
0
[1 0]


## MultiQubit Class
This class let's you construct a tensor product of multiple qubits, used later as an input to the quantum circuit.
Let's construct a MultiQubit object:

In [96]:
from multi_qubit import MultiQubit
mt = MultiQubit()

Now let's add a few qubits to this tensor product:

In [97]:
mt.add_qubit(q0)
mt.add_qubit(q1)
mt.add_qubit(q1)

Again we can print the tensor product of multiple qubits in tensor form and in vector form:



In [98]:
mt.print_tensor_form()
mt.print_vector_form()

Tensor product in basis state form: |011⟩
The vector of the tensor product is: [0 0 0 1 0 0 0 0]


This class also has getters for the tensor product vector and the number of qubits in the tensor product:

In [99]:
print(mt.get_number_of_qubits())
print(mt.get_tensor_vector())

3
[0 0 0 1 0 0 0 0]


## Quantum Circuit Class
The QuantumCircuit class allows for building a quantum circuit. By adding and removing a quantum gates on every qubit at each vertical and horizontal axis. Every iteration from left to right is described by layers. Each layer is compromised of a tensor product of single qubit gates or controlled gates or swap gates.

Let's construct a quantum circuit object with 3 qubits and add a NOT gate to the first qubit. 
The constructer recieves the number of qubits in the circuit. Then we use add_single_qubit_gate method which recieves target_qubit, layer_index, gate type and phase as arguments. Gate types are: I - identity gate, X - X Gate, Y - Y gate, Z - Z gate, H - Hadamard gate.

Note that the target index and layer index are counted from 0.

In [100]:
from circuit import QuantumCircuit
circuit = QuantumCircuit(number_of_qubits=3)
circuit.add_single_qubit_gate(0,0,"X")

Now let's print the circuit to see the construction:

In [101]:
circuit.print_circuit()


Circuit Diagram:
q0: ─[X]──
q1: ──────
q2: ──────


The quantum circuit class initializes a two dimensional array. Every row of the array corresponds to each layer and every column to every gates in that layer. When we apply the circuit on an input or want to print the total unitary operation on the input state, the class computes all of the gate matrices into one unitary matrix.

Let's print this circuit's final matrix:

In [102]:
circuit.print_operator_matrix()

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


Now let's introduce how we can apply the circuit on an input state. Note that the input state which is a tensor product must have the same number of qubits as the circuit otherwise a ValueError will be raised.

In [103]:
result = circuit.apply_circuit(mt)
result.print_tensor_form()

Tensor product in basis state form: |111⟩


We took the 011 state and got the 111 state which is what we expect to see with a NOT gate acting on a qubit.

If we want to reset the circuit to the intial state we can use the following method:

In [104]:
circuit.reset_circuit()
circuit.print_circuit()


Circuit Diagram:
q0: ──────
q1: ──────
q2: ──────


The circuit is now reset as expected.

Now let's introduce controlled gates. To add a controlled gate, use the add_controlled_qubit_gate method and we apply on the input state:

In [105]:
circuit.add_controlled_qubit_gate(0, 0, 1, 'X')  # Add a controlled-X gate with control on qubit 0 and target qubit 1 in layer 0
circuit.print_circuit()

# The input state:
mt.print_tensor_form()
result = circuit.apply_circuit(mt)
# Output state
result.print_tensor_form()


Circuit Diagram:
q0: ─[CX]─
q1: ──●0──
q2: ──────
Tensor product in basis state form: |011⟩
Tensor product in basis state form: |111⟩


The number after the circle shows the qubit index to which it's connected. This is used for situations where many control qubits are in the same layer for easier readability.

Now let's introduce the swap gates which are constructed in a similar fashion such as the controlled qubit gate.

In [106]:
circuit.reset_circuit()
circuit.add_swap_gate(0,2,0)
circuit.print_circuit()

# The input state:
mt.print_tensor_form()
result = circuit.apply_circuit(mt)
# Output state
result.print_tensor_form()


Circuit Diagram:
q0: ──⨉2──
q1: ──────
q2: ──⨉0──
Tensor product in basis state form: |011⟩
Tensor product in basis state form: |110⟩


Note that when given indexes with an already gate assigned to that index, a ValueError will be raised.

We can add layers to this quantum circuit and simillary also remove layers using these methods:

In [107]:
circuit.add_layer()
circuit.add_single_qubit_gate(0,1,"Y")
circuit.print_circuit()

# The input state:
mt.print_tensor_form()
result = circuit.apply_circuit(mt)
# Output state
result.print_tensor_form()


Circuit Diagram:
q0: ──⨉2───[Y]──
q1: ────────────
q2: ──⨉0────────
Tensor product in basis state form: |011⟩
Tensor product in basis state form: 1j|111⟩


And remove layers:


In [108]:
circuit.remove_last_layer()
circuit.print_circuit()


Circuit Diagram:
q0: ──⨉2──
q1: ──────
q2: ──⨉0──


To remove specific two qubit gate we can use the following method:

In [109]:
circuit.remove_two_qubit_gate(0,2,0)
circuit.print_circuit()


Circuit Diagram:
q0: ──────
q1: ──────
q2: ──────


Now let's add a single qubit gate with a phase and show how to remove it:

In [110]:
circuit.add_single_qubit_gate(0,0,"P", np.pi)
circuit.print_circuit()

circuit.remove_single_qubit_gate(0,0)
circuit.print_circuit()


Circuit Diagram:
q0: ─[P]──
q1: ──────
q2: ──────

Circuit Diagram:
q0: ──────
q1: ──────
q2: ──────


Now for the crown jewel of this library the Quantum Fourier Transform. The library has built in preset of qft circuit by the number of qubits:

In [111]:
circuit.load_qft_preset()
circuit.print_circuit()

# The input state:
mt.print_tensor_form()
result = circuit.apply_circuit(mt)
# Output state
result.print_tensor_form()


Circuit Diagram:
q0: ─[H]───[CP]──[CP]─────────────────────⨉2──
q1: ────────●0─────────[H]───[CP]─────────────
q2: ──────────────●0──────────●1───[H]────⨉0──
Tensor product in basis state form: |011⟩
Tensor product in basis state form: 0.3536j|000⟩ - (0.25+0.25j)|001⟩ + 0.3536|010⟩ - (0.25-0.25j)|011⟩ + 0.3536j|100⟩ - (0.25+0.25j)|101⟩ + 0.3536|110⟩ - (0.25-0.25j)|111⟩


The qft matrix we see is:

In [112]:
circuit.print_operator_matrix()

[[ 0.        -3.53553391e-01j  0.        +3.53553391e-01j
   0.        -3.53553391e-01j  0.        +3.53553391e-01j
   0.        -3.53553391e-01j  0.        +3.53553391e-01j
   0.        -3.53553391e-01j  0.        +3.53553391e-01j]
 [ 0.        -3.53553391e-01j -0.25      +2.50000000e-01j
   0.35355339-2.16489014e-17j -0.25      -2.50000000e-01j
   0.        +3.53553391e-01j  0.25      -2.50000000e-01j
  -0.35355339+2.16489014e-17j  0.25      +2.50000000e-01j]
 [ 0.        -3.53553391e-01j -0.35355339+2.16489014e-17j
   0.        +3.53553391e-01j  0.35355339-2.16489014e-17j
   0.        -3.53553391e-01j -0.35355339+2.16489014e-17j
   0.        +3.53553391e-01j  0.35355339-2.16489014e-17j]
 [ 0.        -3.53553391e-01j -0.25      -2.50000000e-01j
  -0.35355339+2.16489014e-17j -0.25      +2.50000000e-01j
   0.        +3.53553391e-01j  0.25      +2.50000000e-01j
   0.35355339-2.16489014e-17j  0.25      -2.50000000e-01j]
 [ 0.        +3.53553391e-01j  0.        +3.53553391e-01j
   0.     

This is unreadable because of floating inaccuracies and large matrix.
In such a case the library implements tests that compare the output matrices of computed circuits with the matrices from theory. 
Using this formula for every element i,j in the expected matrix:

`N = 2**num_qubits`

`expected_qft_matrix[i,j] = (1/np.sqrt(N)) * np.exp(2j * np.pi * i * j / (N))`

This function compares between the ouput matrices of this library and theory on 2 to 10 qubits as input and outputs the runtime:

In [113]:
from circuit_test import test_qft_matrix_output

test_qft_matrix_output()

======== QFT Test runtime: 7.55 seconds =========
=============== QFT tests passed! ===============


Lastly, here's an example of a qft circuit on a 5 qubit system, showing that the qft circuit can be loaded on any number of qubits:

In [114]:
q0 = Qubit(1,0)
q1 = Qubit(0,1)
mt = MultiQubit()
mt.add_qubit(q1)
mt.add_qubit(q0)
mt.add_qubit(q0)
mt.add_qubit(q1)
mt.add_qubit(q1)

circuit = QuantumCircuit(number_of_qubits=5)
circuit.load_qft_preset()
circuit.print_circuit()
print("Input State:")
mt.print_tensor_form()
result = circuit.apply_circuit(mt)
print("Output State:")
result.print_tensor_form()


Circuit Diagram:
q0: ─[H]───[CP]──[CP]──[CP]──[CP]───────────────────────────────────────────────────────────────⨉4──
q1: ────────●0─────────────────────[H]───[CP]──[CP]──[CP]───────────────────────────────────────⨉3──
q2: ──────────────●0──────────────────────●1───────────────[H]───[CP]──[CP]─────────────────────────
q3: ────────────────────●0──────────────────────●1────────────────●2─────────[H]───[CP]─────────⨉1──
q4: ──────────────────────────●0──────────────────────●1────────────────●2──────────●3───[H]────⨉0──
Input State:
Tensor product in basis state form: |10011⟩
Output State:
Tensor product in basis state form: 0.1768|00000⟩ - (0.147+0.09821j)|00001⟩ + (0.06765+0.1633j)|00010⟩ + (0.03449-0.1734j)|00011⟩ - (0.125-0.125j)|00100⟩ + (0.1734-0.03449j)|00101⟩ - (0.1633+0.06765j)|00110⟩ + (0.09821+0.147j)|00111⟩ - 0.1768j|01000⟩ - (0.09821-0.147j)|01001⟩ + (0.1633-0.06765j)|01010⟩ - (0.1734+0.03449j)|01011⟩ + (0.125+0.125j)|01100⟩ - (0.03449+0.1734j)|01101⟩ - (0.06765-0.1633j)|0111